<a href="https://colab.research.google.com/github/canislatranscoxus/gcp_ml_certification/blob/master/4a_sample_babyweight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Babyweight - sample data.

Canis notes.

this is my practice notebook, using as a reference

https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/structured/solutions/4a_sample_babyweight.ipynb

### import libraries

In [3]:
%%bash
sudo pip freeze | grep google-cloud-bigquery

google-cloud-bigquery==1.21.0
google-cloud-bigquery-storage==1.1.0


In [2]:
from google.cloud import bigquery
import pandas as pd


In [4]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: 


In [5]:
PROJECT = "my_project"  # Replace with your PROJECT

### declare some variables

In [3]:
modulo_divisor = 10
train_percent  = 8
eval_percent   = 1


print( 'modulo_divisor: {}'.format( modulo_divisor ) )
print( 'train_percent : {}'.format( train_percent  ) )
print( 'eval_percent  : {}'.format( eval_percent   ) )


modulo_divisor: 10
train_percent : 8
eval_percent  : 1


In [ ]:
# COLUMNS


source_year
year
month
day
wday
state
is_male
child_race
weight_pounds
plurality
apgar_1min
apgar_5min
mother_residence_state
mother_race
mother_age
gestation_weeks
lmp
mother_married
mother_birth_state
cigarette_use
cigarettes_per_day
alcohol_use
drinks_per_week
weight_gain_pounds
born_alive_alive
born_alive_dead
born_dead
ever_born
father_race
father_age
record_weight

### recipe for splitting is
80 % training


10 % eval

10 % test

In [4]:
training_query = '''
SELECT *
FROM table
WHERE
      abs( mod( farm_fingerprint( cast( my_date_column as string) ) ) ) < 8
'''

test_query = '''
SELECT *
FROM table
WHERE
      abs( mod( farm_fingerprint( cast( my_date_column as string) ) ) ) = 8
'''



In [14]:
sql_base = ''' WITH babyweight AS
SELECT 
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year  AS STRING),
            CAST(month AS STRING),
            CAST(date  AS STRING)
        )
    ) AS hash_values
FROM 
      `bigquery-public-data.samples.natality`  
WHERE 
    year                > 2000
    AND weight_pounds   > 0
    AND mother_age      > 0
    AND plurality       > 0
    AND gestation_weeks > 0

SELECT *
FROM   babyweight
WHERE  
       abs(mod( hash_values, {} )) 
'''.format( modulo_divisor )


In [10]:
sql_base

' WITH babyweight AS\nSELECT \n    weight_pounds,\n    is_male,\n    mother_age,\n    plurality,\n    gestation_weeks,\n    FARM_FINGERPRINT(\n        CONCAT(\n            CAST(year  AS STRING),\n            CAST(month AS STRING),\n            CAST(date  AS STRING)\n        )\n    ) AS hash_values\nFROM \n      `bigquery-public-data.samples.natality`  \nWHERE \n    year                > 2000\n    AND weight_pounds   > 0\n    AND mother_age      > 0\n    AND plurality       > 0\n    AND gestation_weeks > 0\n\nSELECT *\nFROM   babyweight\nWHERE  \n       abs(mod( hash_values, 10 )) \n'

In [16]:
sql_train = '{sql_base} < {train_percent} '.format( sql_base=sql_base, train_percent=train_percent )
sql_eval  = '{sql_base} = {train_percent} '.format( sql_base=sql_base, train_percent=train_percent )


In [17]:
sql_train



' WITH babyweight AS\nSELECT \n    weight_pounds,\n    is_male,\n    mother_age,\n    plurality,\n    gestation_weeks,\n    FARM_FINGERPRINT(\n        CONCAT(\n            CAST(year  AS STRING),\n            CAST(month AS STRING),\n            CAST(date  AS STRING)\n        )\n    ) AS hash_values\nFROM \n      `bigquery-public-data.samples.natality`  \nWHERE \n    year                > 2000\n    AND weight_pounds   > 0\n    AND mother_age      > 0\n    AND plurality       > 0\n    AND gestation_weeks > 0\n\nSELECT *\nFROM   babyweight\nWHERE  \n       abs(mod( hash_values, 10 )) \n < 8 '

### export from BigQuery to Dataframe

In [ ]:
df_train = bg.query( sql_train ).to_dataframe()
df_eval  = bq.query( sql_eval  ).to_dataframe()
